# 第5回講義 演習

## 課題 2. 文章の属性を制御して文章を生成するモデルの実装

### モデルの概要
前回の課題のVAE言語モデルでは文章を生成することはできましたが, 文章の属性 (極性, 時制, 長さ, 丁寧さ, etc.) は制御することができませんでいた.

今回の課題では, Z. Hu et al. (2017) で提案された, **属性を制御しながら文章を生成することのできるモデル**を実装します.
モデルの全体像は次の通りになっています.

<img src="figure/hu_icml2017.png">

出典: Z. Hu et al. "Toward Controlled Generation of Text", ICML, 2017

Encoder, GeneratorはVAE言語モデルのものとほぼ同じです (Generatorは前課題のDecoderと同じですが, もとの論文に合わせてGeneratorと呼んでいます) が, **Generatorが生成した文の属性に対してフィードバックを行うDiscriminator**が新たに追加されています.

また, 前回のVAE言語モデルでは潜在変数は$z$と一括りに扱っていましたが, ここでは制御したい属性に対する潜在変数を**潜在コード$c$**として$z$から独立させます.
潜在変数$z$は前回と同様ガウス分布によりモデリングしますが, 潜在コード$c$は入力文$x$に紐づけされた属性のラベルをone_hot化して使用します.

Discriminatorは制御したい属性の種類分用意する必要があり, 例えばもとの論文では2つのDiscriminatorを学習させることにより文章の時制と極性を同時に制御できるモデルを構築しています.
本課題では簡単のためにDiscriminatorは1つだけ使用します.

半教師ありのVAEと同じタスク設定ですが, それよりも良い結果がでることが実験で示されています.


### 学習について
学習は2ステップに分かれています.

第1段階では属性のついていないテキストデータに対して通常のVAE (Encoder + Generator) の学習を行います. この段階では属性ラベル$c$は使用せず, 事前分布$p(c)$からサンプリングしたものを使用します.

第2段階で属性付きのテキストデータを利用し, モデルが属性$c$に沿った文を生成できるようにEncoder, Generator, Discriminatorをそれぞれ学習させていきます.

In [ ]:
import os
import re
import math
import time

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pack_padded_sequence

try:
    from utils import Vocab
except ModuleNotFoundError: # iLect環境
    os.chdir('/root/userspace/chap5')
    from utils import Vocab

np.random.seed(34)
torch.manual_seed(34)

In [ ]:
num_epochs = 2
batch_size = 32

embedding_size = 300 # 単語の埋め込み次元数
hidden_size = 300 # LSTMの隠れ層次元数
latent_z_size = 32  # 潜在変数の次元数
latent_c_size = 2   # 潜在コードの次元数
latent_size = latent_z_size + latent_c_size
n_filters = 100 # Discriminator (CNN) のフィルター数

max_length = 11
min_count = 1 # 出現数がMIN_COUNT未満の単語は<UNK>に置き換える

word_drop_rate = 0.5

PAD = 0
BOS = 1
EOS = 2
UNK = 3
PAD_TOKEN = '<PAD>'
UNK_TOKEN = '<UNK>'
BOS_TOKEN = '<S>'
EOS_TOKEN = '</S>'

beta = 0.1
lmd_c = 0.1
lmd_u = 0.1
lmd_z = 0.1

TRAIN_X_PATH = 'data/styletransfer/train_x.txt'
TRAIN_Y_PATH = 'data/styletransfer/train_y.txt'
VALID_X_PATH = 'data/styletransfer/valid_x.txt'
VALID_Y_PATH = 'data/styletransfer/valid_y.txt'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 1. データの読み込み

今回は, Z. Fu et al. (2017) で提案されている商品レビューのデータセットを使用します.
このデータセットでは800,000件のレビューに対して, その内容が肯定的 (1) なのか否定的 (0) なのかがラベル付されています.
このラベルをDiscriminatorで制御する属性として扱います.

演習では系列長9~10のデータ約8万件に絞って使用します.
データはすべてgithub (https://github.com/fuzhenxin/textstyletransferdata) で公開されているので, 興味と時間のある人は見てみてください.

出典: Z. Fu et al. "Style Transfer in Text: Exploration and Evaluation", AAAI (2018)

データセットの中身は次のようになっています.

In [ ]:
! head -5 data/styletransfer/train_x.txt

In [ ]:
! head -5 data/styletransfer/train_y.txt

In [ ]:
def load_data(path, n_data=10e+10):
    data = []
    for i, line in enumerate(open(path, encoding='utf-8')):
        words = line.strip().split()
        data.append(words)
        if i + 1 >= n_data:
            break
    return data

In [ ]:
class DataLoader:
    def __init__(self, data_x, data_y, batch_size, shuffle=True):
        """
        :param data_x: list, 文章 (単語IDのリスト) のリスト
        :param data_y: list, 属性ラベルのリスト
        :param batch_size: int, バッチサイズ
        :param shuffle: bool, サンプルの順番をシャッフルするか否か
        """
        self.data = list(zip(data_x, data_y))
        
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.start_index = 0
        
        self.reset()
    
    def reset(self):
        if self.shuffle:
            self.data = shuffle(self.data)
        self.start_index = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        # ポインタが最後まで到達したら初期化する
        if self.start_index >= len(self.data):
            self.reset()
            raise StopIteration()
        
        # バッチを取得
        batch_x, batch_c = zip(*self.data[self.start_index:self.start_index+self.batch_size])
        
        # 系列長で降順にソート
        batch = sorted(zip(batch_x, batch_c), key=lambda x: len(x[0]), reverse=True)
        batch_x, batch_c = zip(*batch)
        
        # 系列長を取得
        batch_x = [[BOS] + x + [EOS] for x in batch_x]
        batch_x_lens = [len(x) for x in batch_x]
        
        # <S>, </S>を付与 + 短い系列にパディング
        max_length = max(batch_x_lens)
        batch_x = [x + [PAD] * (max_length - len(x)) for x in batch_x]

        # tensorに変換
        batch_x = torch.tensor(batch_x, dtype=torch.long, device=device)
        batch_c = torch.tensor(batch_c, dtype=torch.long, device=device)
        batch_x_lens = torch.tensor(batch_x_lens, dtype=torch.long, device=device)
        
        # ポインタを更新する
        self.start_index += self.batch_size
        
        return batch_x, batch_c, batch_x_lens

In [ ]:
vocab = Vocab({
    PAD_TOKEN: PAD,
    BOS_TOKEN: BOS,
    EOS_TOKEN: EOS,
    UNK_TOKEN: UNK,
}, UNK_TOKEN)

sens_train_X = load_data(TRAIN_X_PATH)
sens_valid_X = load_data(VALID_X_PATH)

vocab.build_vocab(sens_train_X, min_count)

train_X = [vocab.sentence_to_ids(sen) for sen in sens_train_X]
valid_X = [vocab.sentence_to_ids(sen) for sen in sens_valid_X]

train_Y = np.loadtxt(TRAIN_Y_PATH)
valid_Y = np.loadtxt(VALID_Y_PATH)

vocab_size = len(vocab.word2id)
print('語彙数:', vocab_size)
print('学習用データ数:', len(train_X))
print('検証用データ数:', len(valid_X))

In [ ]:
dataloader_train = DataLoader(train_X, train_Y, batch_size)
dataloader_valid = DataLoader(valid_X, valid_Y, batch_size)

### 2. モデルの定義

#### 2.1. 単語の確率分布に対する単語埋め込みベクトルの取得

単語埋め込みベクトルの計算時に, 離散的な単語IDからだけではなく, 連続的な確率分布に対しても計算を行えるよう実装を行います. 実装は単純な行列積として表します.

イメージは下の図のようになります.
通常の単語の埋め込み操作では特定のk番目の単語ベクトルのみを取り出している (左図) のに対し, 各単語の確率の値に対する総和として埋め込みベクトルを表現 (右図) しています.

<img src="figure/continuous_approximation.png">

これにより, モデル内における勾配の伝播が行いやすくなります.

#### 2.2. Encoder

VAE言語モデルで実装したものとほぼ同じですが, 確率分布に対しても単語埋め込みベクトルの計算を行えるよう実装を修正します.

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, latent_z_size):
        """
        :param vocab_size: int, 入力単語の語彙数
        :param embedding_size: int, 単語埋め込みの次元数
        :param hidden_size: int, LSTMの隠れ層の次元数
        :param latent_z_size: int, 潜在変数zの次元数
        """
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers=1, batch_first=True)
        self.linear_m  = nn.Linear(hidden_size, latent_z_size)
        self.linear_v  = nn.Linear(hidden_size, latent_z_size)
    
    def forward(self, x, x_lens=None):
        """
        :param x: tensor, 単語id列のバッチ or 単語の確率分布列のバッチ, size=(バッチサイズ, 系列長) or (バッチサイズ, 系列長, 語彙数)
        :param x_lens: tensor, 単語id列の長さのバッチ, size=(バッチサイズ, 系列長)
        :return z: tensor, サンプリングした潜在変数, size=(バッチサイズ, latent_size)
        :return mean: tensor, 潜在変数の平均, size=(バッチサイズ, latent_size)
        :return lvar: tensor, 潜在変数のlog分散, size=(バッチサイズ, latent_size)
        :return loss_kl: tensor, KLダイバージェンス, size=()
        """
        if x.dim() == 2: # xが単語IDのtensorの場合
            x = self.embedding(x)
            if x_lens is not None:
                x = pack_padded_sequence(x, lengths=x_lens, batch_first=True) # <PAD>の部分は無視してエンコードする
        elif x.dim() == 3: # xが単語の確率分布のtensorの場合
            x = # WRITE ME

        _, (h_T, _) = self.lstm(x) # LSTM
        
        mean = self.linear_m(h_T[0]) # 平均
        lvar = self.linear_v(h_T[0]) # 分散のlog
        std = torch.exp(0.5 * lvar) # 標準偏差
        
        eps = torch.randn(mean.size()).to(device) # 標準正規分布からサンプリング
        z = mean + std * eps # 潜在変数
        
        loss_kl = - 0.5 * torch.mean(torch.sum(1 + lvar - mean**2 - lvar.exp(), dim=1)) # KLダイバージェンスの計算

        return z, mean, lvar, loss_kl

#### 2.3. Generator

VAE言語モデルで実装したものとほぼ同じですが, Encoderと同様に, 確率分布に対しても単語埋め込みベクトルの計算を行えるよう実装を修正します.

In [ ]:
class Generator(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, latent_size, word_drop_rate):
        """
        :param vocab_size: int, 入力単語の語彙数
        :param embedding_size: int, 単語埋め込みの次元数
        :param hidden_size: int, LSTMの隠れ層の次元数
        :param latent_size: int, 潜在変数zの次元数
        :param word_drop_rate: int, 入力単語をunkに置き換える確率
        """
        super(Generator, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size+latent_size, hidden_size, num_layers=1, batch_first=True)
        self.linear_h = nn.Linear(latent_size, hidden_size)
        self.linear_c = nn.Linear(latent_size, hidden_size)
        self.out = nn.Linear(hidden_size, vocab_size)

        self.vocab_size = vocab_size
        self.word_drop_rate = word_drop_rate

    def forward(self, x, zc):
        """
        :param x: tensor, 入力単語id列のバッチ, size=(バッチサイズ, 系列長)
        :param zc: tensor, 潜在変数と潜在コードのバッチ, size=(バッチサイズ, latent_z_size+latent_size)
        :return y: tensor, Decoderの出力, size=(バッチサイズ, 系列長, 語彙数)
        """
        # 初期状態
        h_0 = self.linear_h(zc).unsqueeze(0) # 隠れ層の初期状態, size=(1, バッチサイズ, 隠れ層の次元数)
        c_0 = self.linear_c(zc).unsqueeze(0) # cellの初期状態, size=(1, バッチサイズ, 隠れ層の次元数)
        
        # 単語を一定確率pで<unk>に置き換える
        if self.training:
            prob = torch.full_like(x, self.word_drop_rate, dtype=torch.float32).to(device)
            mask = torch.bernoulli(prob)
            x = torch.where(mask == 1, torch.full_like(x, UNK).to(device), x)
        
        x = self.embedding(x) # 単語の埋め込み
        x = torch.cat([x, zc.unsqueeze(1).repeat(1, x.shape[1], 1)], dim=2) # 潜在変数+コードzcを毎時刻単語のembeddingにconcatする
        
        h, (_, _) = self.lstm(x, (h_0, c_0)) # LSTM
        
        y = self.out(h) # size=(バッチサイズ, 系列長, 語彙数)
        
        return y # size=(バッチサイズ, 系列帳, 語彙数)
    
    def sample(self, zc, max_length, soft_decoding, greedy_decoding):
        """
        :param zc: tensor, 潜在変数と潜在コードのバッチ, size=(バッチサイズ, latent_size)
        :param max_length: int, 生成文の最大長
        :param soft_decoding: bool, 各時刻の出力を単語IDとするか単語の確率分布とするか
        :param greedy_decoding: bool, 各時刻で確率の最も高い単語を取得するか確率分布からサンプリングするか
        :return x_hat: tensor, 生成文, size=(バッチサイズ, 系列長)
        """
        # 初期状態
        h_0 = self.linear_h(zc).unsqueeze(0) # 隠れ層の初期状態, size=(1, バッチサイズ, 隠れ層の次元数)
        c_0 = self.linear_c(zc).unsqueeze(0) # cellの初期状態, size=(1, バッチサイズ, 隠れ層の次元数)
        
        # 最初の単語は<S>
        x_0 = torch.full((zc.size(0), 1), BOS, dtype=torch.long).to(device) # size=(バッチサイズ, 1)

        if soft_decoding:
            # 単語IDをone-hot化, size=(バッチサイズ, 語彙数)
            x_0 = torch.zeros((x_0.size(0), self.vocab_size), device=device).scatter(1, x_0, 1.0)
    
        zc = zc.unsqueeze(1) # size=(バッチサイズ, 1, latent_size)
        
        x_tm1, h_tm1, c_tm1 = x_0, h_0, c_0

        x_generated = [] # 生成文を格納するlist
        
        if not soft_decoding:
            flag = np.zeros(x_0.size(0), dtype=bool) # 出力が終わったかどうかのFlag
        
        for _ in range(max_length):
            if soft_decoding: # 単語の確率分布から埋め込みベクトルを取得
                x_t = torch.matmul(x_tm1, self.embedding.weight).unsqueeze(1) # size=(バッチサイズ, 1, embedding_size)
            else: # 単語IDから埋め込みベクトルを取得
                x_t = self.embedding(x_tm1) # size=(バッチサイズ, 1, embedding_size)
            
            x_t = torch.cat([x_t, zc], dim=2) # 潜在変数+潜在コードと単語埋め込みベクトルを連結, size=(バッチサイズ, 1, embedding_size+latent_size)
            
            _, (h_t, c_t) = self.lstm(x_t, (h_tm1, c_tm1)) # LSTM
            
            y_t = F.softmax(self.out(h_t[0]), dim=-1) # Softmax, size=(バッチサイズ, 語彙数)
            
            if not soft_decoding:
                if greedy_decoding: # 確率の一番高い単語を取得する
                    y_t = y_t.argmax(1).unsqueeze(1)
                else: # 確率分布からサンプリングする
                    y_t = torch.multinomial(y_t, 1) # (バッチサイズ, 1)
            
            x_generated.append(y_t)

            # t -> t-1
            x_tm1, h_tm1, c_tm1 = y_t, h_t, c_t

            if not soft_decoding:
                # </S>が出力されたらFlagを更新する
                flag_t = (y_t.squeeze().cpu().numpy() == EOS)
                flag = np.logical_or(flag, flag_t)

                # Bすべての系列で</S>が出力されたら終了
                if np.all(flag):
                    break

        # listからtensorに変換する
        if soft_decoding:
            x_generated = torch.stack(x_generated, dim=1)
        else:
            x_generated = torch.cat(x_generated, dim=1)

        return x_generated # size=(バッチサイズ, 系列長, 語彙数) or (バッチサイズ, 系列長)

#### 2.4. Discriminator

Generatorから生成された文の属性を判別するDisciriminatorをCNNで実装します.
これはChapter 4で実装したものと同じです.

これに関しても確率分布に対する単語埋め込みベクトルの計算を追加で実装します.

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, vocab_size, embedding_size, n_filters, latent_c_size):
        super(Discriminator, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.conv_1 = nn.Conv1d(embedding_size, n_filters, kernel_size=3)
        self.conv_2 = nn.Conv1d(embedding_size, n_filters, kernel_size=4)
        self.conv_3 = nn.Conv1d(embedding_size, n_filters, kernel_size=5)
        self.out = nn.Linear(n_filters*3, latent_c_size)

    def forward(self, x):
        """
        :param x: tensor, 単語id列のバッチ or 単語の確率分布列のバッチ, size=(バッチサイズ, 系列長) or (バッチサイズ, 系列長, 語彙数)
        :return x: tensor, size=(バッチサイズ, latent_c_size)
        """
        x_len = x.size(1)

        if x.dim() == 2: # xが単語IDのtensorの場合
            x = self.embedding(x)
        elif x.dim() == 3: # xが単語の確率分布のtensorの場合
            x = torch.matmul(x, self.embedding.weight)

        x = x.permute(0, 2, 1) # conv用に次元を入れ替え, size=(バッチサイズ, embedding_size, 系列長)

        x1 = torch.tanh(self.conv_1(x)) # フィルター1に対して畳み込み, size=(バッチサイズ, フィルター数, 系列長-2)
        x2 = torch.tanh(self.conv_2(x)) # フィルター1に対して畳み込み, size=(バッチサイズ, フィルター数, 系列長-3)
        x3 = torch.tanh(self.conv_3(x)) # フィルター1に対して畳み込み, size=(バッチサイズ, フィルター数, 系列長-4)

        x1 = F.max_pool1d(x1, x_len-2) # x1に対してpooling, size=(バッチサイズ, フィルター数, 1)
        x2 = F.max_pool1d(x2, x_len-3) # x2に対してpooling, size=(バッチサイズ, フィルター数, 1)
        x3 = F.max_pool1d(x3, x_len-4) # x3に対してpooling, size=(バッチサイズ, フィルター数, 1)

        x = torch.cat([x1, x2, x3], dim=1)[:, :, 0] # size=(バッチサイズ, フィルター数*3)
        x = self.out(x)

        return x # (バッチサイズ, latent_c_size)

In [ ]:
E_args = {
    'vocab_size': vocab_size,
    'embedding_size': embedding_size,
    'hidden_size': hidden_size,
    'latent_z_size': latent_z_size
}

G_args = {
    'vocab_size': vocab_size,
    'embedding_size': embedding_size,
    'hidden_size': hidden_size,
    'latent_size': latent_size,
    'word_drop_rate': word_drop_rate
}

D_args = {
    'vocab_size': vocab_size,
    'embedding_size': embedding_size,
    'n_filters': n_filters,
    'latent_c_size': latent_c_size
}

In [ ]:
E = Encoder(**E_args).to(device) # cudnnのエラーが出る場合はこのセルをもう一度実行してください
G = Generator(**G_args).to(device)
D = Discriminator(**D_args).to(device)

optimizer_E = optim.Adam(E.parameters())
optimizer_G = optim.Adam(G.parameters())
optimizer_D = optim.Adam(D.parameters())

#### 2.5. その他の関数

In [ ]:
def sample_z_prior(batch_size):
    """事前分布p(z)からzをサンプリングする
    :param batch_size: int, バッチサイズ
    :return z: tensor, サンプリングされたz, size=(バッチサイズ, latent_z_size)
    """
    z = torch.randn(batch_size, latent_z_size, device=device)
    return z

In [ ]:
def sample_c_prior(batch_size):
    """事前分布p(c)からcをサンプリングする
    :param batch_size: int, バッチサイズ
    :return c: tensor, サンプリングされたc, size=(バッチサイズ, latent_c_size)
    """
    weights = torch.ones(latent_c_size, dtype=torch.float).to(device)
    c = torch.multinomial(weights, batch_size, replacement=True)
    c = torch.eye(latent_c_size)[c].to(device) # one_hot化
    return c

### 3. 学習

#### 3.1. 損失関数の定義

全部で5つの損失関数を実装します.

##### 3.1.1. $\mathcal{L}_{VAE}$

VAE言語モデルの損失関数と同じです. 係数$\lambda$でKL項の強さをコントロールします.

第1段階, 第2段階の学習で使用します.

$$
\mathcal{L}_{VAE}(\theta_G, \theta_E; x) = -\mathbb{E}_{q_E(z|x) q_D(c|x)} \left[\log p_G(x|z, c)\right] + \lambda \cdot D_{\mathrm{KL}}\left(q_E(z|x)||p(z)\right)
$$

<img src="figure/loss_vae.png" width="800mm">

In [ ]:
def compute_loss_vae(x, x_lens, lmd, use_c_prior=True, is_train=False):
    """VAEの損失関数 (負の対数尤度 + lmd * KLダイバージェンス) を計算する
    :param x: tensor, 入力単語id列のバッチ, size=(バッチサイズ, 系列長)
    :param x_len: tensor, 入力単語の系列長のバッチ, size=(バッチサイズ, 系列長)
    :param lmd: int, KLダイバージェンスの大きさをコントロールする係数
    :param use_c_prior: bool, 潜在コードcを事前分布p(c)からサンプリングする か Discriminatorからの出力を利用するか
    :param is_train: bool, モデル (EとD) のパラメータを更新するか否か
    :return loss_vae: tensor, VAEの損失, size=()
    :return nll: tensor, 負の対数尤度, size=()
    :return loss_kl: tensor, KLダイバージェンス, size=()
    """
    # Encoder
    input_encoder = x[:, :-1] # [<S>, x_1, ..., x_T]
    input_encoder_lens = x_lens - 1
    z, mean, lvar, loss_kl = E.forward(input_encoder, input_encoder_lens)
    
    if use_c_prior:
        c = sample_c_prior(x.size(0))
    else:
        c = F.softmax(D.forward(input_encoder), dim=-1)
    
    zc = torch.cat([z, c], dim=1)
    
    # Generator
    input_generator = x[:, :-1] # [<S>, x_1, ..., x_T]
    target_generator = x[:, 1:].contiguous() # [x_1, x_2, ..., </S>]
    output_generator = G.forward(input_generator, zc)
    
    # 損失
    nll_all = F.cross_entropy(output_generator.view(-1, vocab_size), target_generator.view(-1), ignore_index=PAD, size_average=False, reduce=False)
    nll_mb = torch.sum(nll_all.view(output_generator.size(0), output_generator.size(1)), dim=1)
    nll = nll_mb.mean()
    
    loss_vae = nll + lmd * loss_kl
    
    if is_train:
        E.zero_grad()
        G.zero_grad()
        loss_vae.backward()
        optimizer_E.step()
        optimizer_G.step()
    
    return loss_vae, nll, loss_kl

##### 3.1.2. $\mathcal{L}_s(\theta_D)$

教師ありデータ ($x_L, c_L$) に対するDiscriminatorの識別損失を表します.

第2段階の学習で使用します.

$$
    \mathcal{L}_s(\theta_D) = -\mathbb{E}_{\mathcal{X}_L}\left[\log q_D(c_L|x_L)\right]
$$

<img src="figure/loss_s.png" width="800mm">

In [ ]:
def compute_loss_s(x, c):
    """
    :param x: tensor, 入力単語id列のバッチ, size=(バッチサイズ, 系列長)
    :param c: tensor, 潜在コードcのバッチ, size=(バッチサイズ, latent_c_size)
    :return loss_s: tensor, size=()
    """
    input_encoder = x[:, 1:-1] # [x_1, x_2, ..., x_T]
    c_pred = D.forward(input_encoder) # (バッチサイズ, latent_c_size)
    
    loss_s = F.cross_entropy(c_pred, c)
    
    return loss_s

##### 3.1.3. $\mathcal{L}_u(\theta_D)$

Generatorが生成した文に対するDiscriminatorの識別損失を表します.

第2段階の学習で使用します.

$$
    \mathcal{L}_u(\theta_D) = -\mathbb{E}_{p_G(\hat{x}|z,c)p(z)p(c)}\left[\log q_D(c|\hat{x}) + \beta\mathcal{H}\left(q_D(c'|\hat{x})\right)\right]
$$

<img src="figure/loss_u.png" width="800mm">

In [ ]:
def compute_loss_u():
    """
    :return loss_u: tensor, size=()
    """
    z = sample_z_prior(batch_size)
    c = sample_c_prior(batch_size)
    zc = torch.cat([z, c], dim=1)
    
    x_hat = G.sample(zc, max_length, soft_decoding=False, greedy_decoding=False) # (バッチサイズ, 系列長)
    
    c_pred = D.forward(x_hat) # (バッチサイズ, latent_c_size)
    
    loss_u = F.cross_entropy(c_pred, c.argmax(dim=1)) - beta * F.log_softmax(c_pred, dim=-1).mean()
    
    return loss_u

##### 3.1.4. $\mathcal{L}_{Attr, c}(\theta_G)$

事前分布からサンプリングされた属性 $c \sim p(c)$ に対し, Generatorがどれだけそれに沿った文を生成できたかを表します.

第2段階の学習で使用します.

$$
    \mathcal{L}_{Attr, c}(\theta_G) = -\mathbb{E}_{p(z)p(c)}\left[\log q_D\left(c|\tilde{G}_{\tau(z,c)}\right)\right]
$$

<img src="figure/loss_attr_c.png" width="800mm">

##### 3.1.5. $\mathcal{L}_{Attr, z}(\theta_G)$

サンプリングされた潜在変数 $z \sim p(z)$に対し, Generatorがどれだけそれに沿って文を生成できたかを表します.

第2段階の学習で使用します.

$$
    \mathcal{L}_{Attr, z}(\theta_G) = \mathbb{E}_{p(z)p(c)}\left[\mathrm{MSE}(z, q_E\left(z|\tilde{G}_{\tau}(z,c)\right))\right]
$$

<img src="figure/loss_attr_z.png" width="800mm">

In [ ]:
def compute_loss_attr():
    """
    :return loss_attr_c: tensor, size=()
    :return loss_attr_z: tensor, size=()
    """
    z = sample_z_prior(batch_size)
    c = sample_c_prior(batch_size)
    zc = torch.cat([z, c], dim=1)

    # Generator
    x_tilde = G.sample(zc, max_length, soft_decoding=True, greedy_decoding=False)

    # Discriminator
    c_pred = D.forward(x_tilde)
    loss_attr_c = F.cross_entropy(c_pred, torch.argmax(c, dim=1))

    # Encoder
    z_pred, _, _, _ = E.forward(x_tilde)
    loss_attr_z = F.mse_loss(z_pred, z)
    
    return loss_attr_c, loss_attr_z

#### 3.2. VAE (Encoder + Generator) の学習

第1段階では属性のついていないテキストデータに対して通常のVAE (Encoder + Generator) の学習を行います. この段階では属性ラベル$c$は使用せず, 事前分布$p(c)$からサンプリングしたものを使用します.

iLect環境で1epochあたり2分強かかります

In [ ]:
def get_kl_weight(step):
    """step数でアニーリングしたKL項の重みを取得する (0 -> 1)
    :param step: int, 累積学習ステップ数
    :return : float
    """
    return (math.tanh((step - 3500)/1000) + 1) /2

In [ ]:
step = 0
start_time = time.time()
for epoch in range(num_epochs):
    # Train
    E.train()
    G.train()
    nll_train = []
    kl_train = []
    for batch_x, _, batch_x_lens in dataloader_train:
        lmd = get_kl_weight(step)
        
        loss, nll, loss_kl = compute_loss_vae(batch_x, batch_x_lens, lmd, is_train=True)
        nll_train.append(nll.item())
        kl_train.append(loss_kl.item())
        
        step += 1
        
#         break

    # Valid
    E.eval()
    G.eval()
    nll_valid = []
    kl_valid = []
    for batch_x, _, batch_x_lens in dataloader_valid:
        loss, nll, loss_kl = compute_loss_vae(batch_x, batch_x_lens, lmd, is_train=False)
        nll_valid.append(nll.item())
        kl_valid.append(loss_kl.item())
        
#         break
    
    print('EPOCH: {}, LMD: {:.2f}, Train [NLL: {:.2f}, KL: {:.2f}], Valid [NLL: {:.2f}, KL: {:.2f}], Elapsed Time: {:.2f}[s]'.format(
        epoch + 1,
        lmd,
        np.mean(nll_train),
        np.mean(kl_train),
        np.mean(nll_valid),
        np.mean(kl_valid),
        time.time() - start_time
    ))
    
#     break

学習させたGeneratorを用いて文を生成してみます.

In [ ]:
def sample(batch_size, max_length, c=None):
    """Generatorから文を生成する
    :param batch_size: int, バッチサイズ
    :param max_length: int, 生成文の最大長
    :return x_hat: tensor, 生成文, size=(バッチサイズ, 系列長)
    :return : tensor, 生成文の属性, size=(バッチサイズ)
    """
    z = sample_z_prior(batch_size)
    if c is None:
        c = sample_c_prior(batch_size)
    zc = torch.cat([z, c], dim=1)
    
    x_hat = G.sample(zc, max_length, soft_decoding=False, greedy_decoding=True)
    
    return x_hat, c.argmax(dim=1)

In [ ]:
n_samples = 10

E.eval()
G.eval()

x, _ = sample(n_samples, max_length)
x = x.cpu().numpy()

for i, x_i in enumerate(x):
    x_i = ' '.join([vocab.id2word[i] for i in x_i])
    x_i = re.sub(r' {}.*'.format(EOS_TOKEN), '', x_i)
    print('{}. {}'.format(i, x_i))

#### 3.3. Encoder, Generator, Disciriminatorそれぞれの学習

第2段階で属性付きのテキストデータを利用し, モデルが属性$c$に沿った文を生成できるようにEncoder, Generator, Discriminatorをそれぞれ学習させていきます.

上で定義した損失関数を組み合わせ, それぞれの最終的な損失関数を定義していきます.

##### Discriminatorの損失関数
教師ありデータ$(x_L, c_L)$に対する損失関数と, Generatorが潜在コード$c$の下で生成した文$\tilde{x}$に対する損失関数を組み合わせます.

$$
    \mathcal{L}_D(\theta_D) = \mathcal{L}_s + \lambda_u\mathcal{L}_u
$$

In [ ]:
def compute_loss_D(x, c, is_train=False):
    """
    :param x: tensor, 単語id列のバッチ, size=(バッチサイズ, 系列長)
    :param c: tensor, 潜在コードcのバッチ, size=(バッチサイズ, latent_c_size)
    :param is_train: bool, モデル (D) のパラメータを更新するか否か
    :return loss_D: tensor, size=()
    """
    loss_s = compute_loss_s(x, c)
    loss_u = compute_loss_u()
    
    loss_D = loss_s + lmd_u * loss_u
    
    if is_train:
        D.zero_grad()
        loss_D.backward()
        optimizer_D.step()
    
    return loss_D

##### Generatorの損失関数
VAEの損失関数, 潜在コード$c$に対する損失関数, 潜在変数$z$に対する損失関数を組み合わせます.

$$
    \mathcal{L}_G(\theta_G) = \mathcal{L}_{VAE} + \lambda_c\mathcal{L}_{Attr,c} + \lambda_z\mathcal{L}_{Attr, z}
$$

In [ ]:
def compute_loss_G(x, x_lens, is_train=False):
    """
    :param x: tensor, 単語id列のバッチ, size=(バッチサイズ, 系列長)
    :param x_lens: tensor, 系列長のバッチ, size=(バッチサイズ, 系列長)
    :param is_train: bool, モデル (G) のパラメータを更新するか否か
    :return loss_G: tensor, size=()
    :return loss_vae: tensor, size=()
    :return loss_attr_c: tensor, size=()
    :return loss_attr_z: tensor, size=()
    """
    loss_vae, _, _ = compute_loss_vae(x, x_lens, lmd=1, use_c_prior=False, is_train=False)
    loss_attr_c, loss_attr_z = compute_loss_attr()

    loss_G = loss_vae + lmd_c * loss_attr_c + lmd_z * loss_attr_z
    
    if is_train:
        G.zero_grad()
        loss_G.backward()
        optimizer_G.step()
    
    return loss_G, loss_vae, loss_attr_c, loss_attr_z

##### Encoderの損失関数
VAEの損失関数をそのまま使用します.
この際Encoderのパラメータに対してのみ学習を行い, Generatorのパラメータに対しては行いません.

$$
    \mathcal{L}_{E}(\theta_E) = \mathcal{L}_{VAE}
$$

In [ ]:
def compute_loss_E(x, x_lens, is_train=False):
    """
    :param x: tensor, 単語id列のバッチ, size=(バッチサイズ, 系列長)
    :param x_lens: tensor, 系列長のバッチ, size=(バッチサイズ, 系列長)
    :param is_train: bool, モデル (E) のパラメータを更新するか否か
    :return loss_E: tensor, size=()
    """
    loss_vae, _, _ = compute_loss_vae(x, x_lens, lmd=1, use_c_prior=False, is_train=False)
    
    loss_E = loss_vae
    
    if is_train:
        E.zero_grad()
        loss_E.backward()
        optimizer_E.step()
    
    return loss_E

##### 学習

1epochあたり15分ほどかかります

In [ ]:
start_time = time.time()
for epoch in range(num_epochs):
    # Train
    E.train()
    G.train()
    D.train()
    loss_D_train = []
    loss_G_train = []
    loss_E_train = []
    for batch_x, batch_c, batch_x_lens in dataloader_train:
        # Discriminatorの学習
        loss_D = compute_loss_D(batch_x, batch_c, is_train=True)
        loss_D_train.append(loss_D.item())
        
        # Generatorの学習
        loss_G, _, _, _ = compute_loss_G(batch_x, batch_x_lens, is_train=True)
        loss_G_train.append(loss_G.item())
        
        # Encoderの学習
        loss_E = compute_loss_E(batch_x, batch_x_lens, is_train=True)
        loss_E_train.append(loss_E.item())
        
#         break
    
    # Valid
    E.eval()
    G.eval()
    D.eval()
    loss_D_valid = []
    loss_G_valid = []
    loss_E_valid = []
    
    pred_real_valid = []
    gold_real_valid = []
    pred_gene_valid = []
    gold_gene_valid = []
    for batch_x, batch_c, batch_x_lens in dataloader_valid:
        # Discriminatorの検証
        loss_D = compute_loss_D(batch_x, batch_c, is_train=False)
        loss_D_valid.append(loss_D.item())
        
        c_pred = D.forward(batch_x).argmax(dim=1).tolist()
        c_gold = batch_c.tolist()
        pred_real_valid.extend(c_pred)
        gold_real_valid.extend(c_gold)
        
        x_hat, c_gold = sample(batch_size, max_length)
        c_pred = D.forward(x_hat).argmax(dim=1).tolist()
        c_gold = c_gold.tolist()
        pred_gene_valid.extend(c_pred)
        gold_gene_valid.extend(c_gold)
        
        # Generatorの検証
        loss_G, _, _, _ = compute_loss_G(batch_x, batch_x_lens, is_train=False)
        loss_G_valid.append(loss_G.item())
        
        # Encoderの検証
        loss_E = compute_loss_E(batch_x, batch_x_lens)
        loss_E_valid.append(loss_E.item())
        
#         break
    
    print('''
    EPOCH: {}, Elapsed Time: {:.2f}[s]
    Train [D\'s Loss: {:.2f}, G\'s Loss: {:.2f}, E\'s Loss: {:.2f}]
    Valid [D\'s Loss: {:.2f}, G\'s Loss: {:.2f}, E\'s Loss: {:.2f}, Accuracy for real seq: {:.2f}, Accuracy for generated seq: {:.2f}]
    '''.format(
        epoch + 1,
        time.time() - start_time,
        np.mean(loss_D_train),
        np.mean(loss_G_train),
        np.mean(loss_E_train),
        np.mean(loss_D_valid),
        np.mean(loss_G_valid),
        np.mean(loss_E_valid),
        accuracy_score(gold_real_valid, pred_real_valid),
        accuracy_score(gold_gene_valid, pred_gene_valid),
    ))
    
#     break

##### 指定した属性($c$)で生成

事前分布から $z, c$ をサンプリングしたときに, その属性$c$に沿った文が生成できているか確認してみましょう.

In [ ]:
n_samples = 10

E.eval()
G.eval()

c_neg = torch.eye(2)[torch.zeros(n_samples // 2, dtype=torch.long)].to(device)
c_pos = torch.eye(2)[torch.ones(n_samples // 2, dtype=torch.long)].to(device)

c = torch.cat([c_neg, c_pos], dim=0)

x, c = sample(n_samples, max_length, c)
x = x.cpu().numpy()

for x_i, c_i in zip(x, c):
    x_i = ' '.join([vocab.id2word[j] for j in x_i])
    x_i = re.sub(r' {}.*'.format(EOS_TOKEN), '', x_i)
    print('属性: {}\n生成文: {}\n'.format(c_i, x_i))

参考資料

- 原論文: http://proceedings.mlr.press/v70/hu17e.html
- PyTorch実装 (非公式) : https://github.com/wiseodd/controlled-text-generation